## 04. PyTorch Custom Datasets

=> get own data into pytorch

### 0. Importing and setting up

In [1]:
import torch
from torch import nn

In [2]:
device = "gpu" if torch.cuda.is_available() \
    else "mps" if torch.backends.mps.is_built() else "cpu"
device

'mps'

### 1. Get data

- `Food-101 Dataset:` https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/
    - 101 different classes of food
    - 1000 images per class (750 training)

#### => use a subset of the Food-101 dataset
- use 3 classes of food 
- obly 10% of the images (75 training)

In [3]:
import requests
import zipfile
from pathlib import Path

In [8]:
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

if image_path.is_dir():
    print(image_path, " directory already exists...")
else:
    print(image_path, " does not exist, creating...")
    image_path.mkdir(parents=True, exist_ok=True)


with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading...")
    f.write(request.content)

# unzip the file data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping...")
    zip_ref.extractall(image_path)

data/pizza_steak_sushi  directory already exists...
Downloading...
Unzipping...
